In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
from src.HybridSearchService import HybridSearchService
from src.QueryService import QueryService
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation
from datetime import datetime

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from src.SummaryService import SummaryService

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import pandas as pd

In [5]:
import json

In [6]:
# with open('query_understanding.json', 'r') as f:
#     query_understanding = json.load(f)

# queries_df = pd.DataFrame.from_dict(query_understanding).T.reset_index().rename(columns = {'index': 'query'})

In [6]:
df = pd.read_csv('search_results_5_feb.csv')

In [7]:
import json

In [8]:
df['3'] = df['3'].apply(lambda x: json.loads(x))

In [9]:
from datetime import datetime, timedelta

In [10]:
def get_top_hits_for_query_row(query_row):
    if query_row.finance_query:
        query_date = datetime.strptime(query_row.query_date,'%d/%m/%y') + timedelta(days=1,seconds=-1)
        formatted_date = query_date.strftime('%Y-%m-%dT%H:%M:%S')
        print(query_date, formatted_date)
        if query_row.improved_query:
            # If the date is a future date, then change importance to medium to get articles from ES
            if query_row.recency_importance == 'high' and query_date >= datetime.now():
                results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.improved_query, recency_importance='medium',max_published_date=formatted_date)
            else:
                results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.improved_query, recency_importance=query_row.recency_importance,max_published_date=formatted_date)
        else:
            results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.query, recency_importance='medium', max_published_date=formatted_date)
        top_hits = [x['article_id'] for x in results_object['searchArticleIds']][:10]
        return query_row.name, query_row.query, query_row.query_date, json.dumps(top_hits)
    else:
        return query_row.name, query_row.query, query_row.query_date, json.dumps([])

In [11]:
# a,b,c,d = get_top_hits_for_query_row(queries_df.iloc[5])

In [12]:
def gen_prompt_from_row(row):
    if row['3']:
        dt = (datetime.strptime(row['2'],'%d/%m/%y') + timedelta(days=1,seconds=-1)).strftime('%Y-%m-%dT%H:%M:%S') + '+05:30'
        return SummaryService.create_prompt_for_query_references(search_query=row['1'],references_article_ids=row['3'][:3], today_date=dt)
    else:
        return ''

In [13]:
from openai import OpenAI
client = OpenAI(api_key='')

In [14]:
model = 'gpt-4-1106-preview'

In [15]:
from time import sleep

In [16]:
def fetch_save_responses(query_row):
    try:
        if query_row['3']:
            dt = (datetime.strptime(query_row['2'],'%d/%m/%y') + timedelta(days=1,seconds=-1)).strftime('%Y-%m-%dT%H:%M:%S') + '+05:30'
            try:
                full_prompt = SummaryService.create_prompt_for_query_references(search_query=query_row['1'],references_article_ids=query_row['3'][:3], today_date=dt)
            except:
                print(query_row['3'][:3])
                return False
            res_gpt = client.chat.completions.create(model = model, messages = [{'role': 'system', 'content': ''}, {'role': 'user', 'content': full_prompt}])
            full_response = res_gpt.choices[0].message.content
            cur_df = pd.DataFrame([(query_row['0'],query_row['1'], query_row['2'],query_row['3'], full_prompt, full_response,)])
            cur_df.to_csv('gpt-4-search-responses-5-feb.csv',header=None, mode='a')
            sleep(2)
            return True
    except Exception as e:
        print(f'error {e} for {query_row.query}')
        return False

In [17]:
chunk_size = 10
num_chunks = len(df)//chunk_size + 1

In [18]:
import time

In [19]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [20]:
old_i = 252

In [21]:
old_i

252

In [22]:
for i in range(old_i, num_chunks):
    query_chunk = df.iloc[i*chunk_size:(i+1)*chunk_size]
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(fetch_save_responses, row) for index, row in query_chunk.iterrows()]
        for future in as_completed(futures):
            response = future.result()
    print(f'completed chunk {(i + 1) * chunk_size} queries at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

['65aa564e6391ac19baa86cca', '65b9f00ef3aa2a77a58bf6fc', '65b9f00ef3aa2a77a58bf6f8']['65a91d8b6391ac19baa8322f', '65b4aa001b79671c6b70d118', '65b866c91b79671c6b70d7f2']

['65bb094ff3aa2a77a58bf8d5', '65a3bec2ea435f28caee4cfb', '65adda586391ac19baa871ec']
['65b866621b79671c6b70d749', '65a91c1e6391ac19baa81eaa', '65a91d986391ac19baa832d8']
['65bb0963f3aa2a77a58bfa04', '65bb0977f3aa2a77a58bfbb4', '65ba2847f3aa2a77a58bf7d0']
['65a91d416391ac19baa82e1d', '65a91e756391ac19baa83ec1', '65a91ca06391ac19baa8258c']
['65b633c21b79671c6b70d25a', '658ff8404b25ce4814fb4113', '65b390c61b79671c6b70cf94']
['65a91cb36391ac19baa82699', '65a91e636391ac19baa83dd0', '65a91cb36391ac19baa82692']
completed chunk 2530 queries at Feb 06, 2024 16:42 in 0.23333333333333334 minutes
completed chunk 2540 queries at Feb 06, 2024 16:42 in 0.43333333333333335 minutes
completed chunk 2550 queries at Feb 06, 2024 16:43 in 0.3333333333333333 minutes
completed chunk 2560 queries at Feb 06, 2024 16:43 in 0.48333333333333334 m

In [ ]:
from datasets import Dataset
sys.path.append("../utils")
from pack_dataset import pack_dataset
finetune_id = 'MangoPurpleValley'